Description

In [ ]:
# ZF37 titration with 203bp. Using plasmids to titrate.

## Import Functions

Import Functions and set directories

In [2]:
import functions_paper_1
import importlib
from functions_paper_1 import *
exec(open("functions_paper_1.py").read())
# Local path to Sharepoint (on your computer, change if you are a different user)
datadir = Path(Path('./datadir_srk.txt').read_text())
#datadir = Path(Path('../datadir_yb.txt').read_text())
figure_folder = './figs_2024_promoter_editing_paper/fig3_ZF37titr/fig3_'

In [3]:
display(datadir)

WindowsPath('C:/Users/ChemeGrad2020/OneDrive - Massachusetts Institute of Technology/Documents - GallowayLab')

## Import Data

In [2]:
#Import from other notebooks 
%store -r s_promoters37
s_promoters37 = s_promoters37

%store -r s_promoters37_normEF1a
s_promoters37_normEF1a = s_promoters37_normEF1a

%store -r s_means
s_promoters37_normEF1a_means = s_means

In [ ]:
#Bioreplicates 
# 20240304flow_exp20240301_p3_BioC ('Sneha' attune folder)
# 20240309flow_exp20240306_p1 ('Yunbeen' attune folder)
# 20240309flow_exp20240306_p2 ('Yunbeen' attune folder)
# 20240316flow_exp20240313_p1_ZFtitr ('Sneha' attune folder)
# '20240315flow_exp20240312_p3_ZFtitr' ('Sneha' attune folder)

In [4]:
# Import Data - bioreplicate 1 
folder = '20240304flow_exp20240301_p3_BioC'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data1 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data1['bioreplicate'] = folder 
data1['mCherry-A'] = data1['mRuby2-A']

In [5]:
# Import Data - bioreplicate 2
folder = '20240309flow_exp20240306_p1'
file_path = datadir/'instruments'/'data'/'attune'/'Yunbeen'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Yunbeen'/folder/'well_metadata.yaml' #Assign yaml paths 
data2 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data2['bioreplicate'] = folder 

In [6]:
# Import Data - bioreplicate 3
folder = '20240309flow_exp20240306_p2'
file_path = datadir/'instruments'/'data'/'attune'/'Yunbeen'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Yunbeen'/folder/'well_metadata.yaml' #Assign yaml paths 
data3 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data3['bioreplicate'] = folder 

In [7]:
# Import Data - bioreplicate 4 
folder = '20240316flow_exp20240313_p1_ZFtitr'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data4 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data4['bioreplicate'] = folder

In [8]:
# Import Data - bioreplicate 5 
folder = '20240315flow_exp20240312_p3_ZFtitr'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data5 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data5['bioreplicate'] = folder

Concatenate Data, iRFP670 Gate, Add conditions

In [9]:
data = pd.concat( [data1, data2, data3, data4, data5], ignore_index=True,join='inner')

In [10]:
data = data[ data['mCherry-A'] >0]
data = data[ data['mGL-A'] >0]
data = data[ data['TagBFP-A'] >0]
data = data[ data['iRFP670-A'] >0]
data['log10 mGL-A'] = np.log10(data['mGL-A'])

In [11]:
#Add new condtion 
data['bioreplicate_reporter'] = data['bioreplicate'] + '_' + data['reporter']

#Add total condition 
data['cond'] = data['reporter'] + '.' + str(data['ZF37']) + '.' + data['bioreplicate']

data['ZF37_str'] = data['ZF37'].astype('str')

display(pd.unique(data['bioreplicate_reporter']))

data_unstained_20240304flow_exp20240301_p3_BioC_unstained = data[data['bioreplicate_reporter']=='20240304flow_exp20240301_p3_BioC_unstained']
%store data_unstained_20240304flow_exp20240301_p3_BioC_unstained

array(['20240304flow_exp20240301_p3_BioC_pTET021_203bp 0.1xCre',
       '20240304flow_exp20240301_p3_BioC_pTET021_203bp 0xCre', <NA>,
       '20240304flow_exp20240301_p3_BioC_unstained',
       '20240309flow_exp20240306_p1_pTET021_203bp 0.1xCre',
       '20240309flow_exp20240306_p1_pTET021_203bp 0xCre',
       '20240309flow_exp20240306_p2_pTET021_203bp 0.1xCre',
       '20240309flow_exp20240306_p2_pTET021_203bp 0xCre',
       '20240316flow_exp20240313_p1_ZFtitr_pTET021_203bp 0.1xCre',
       '20240316flow_exp20240313_p1_ZFtitr_pTET021_203bp 0xCre',
       '20240315flow_exp20240312_p3_ZFtitr_pTET021_203bp 0.1xCre',
       '20240315flow_exp20240312_p3_ZFtitr_pTET021_203bp 0xCre'],
      dtype=object)

Stored 'data_unstained_20240304flow_exp20240301_p3_BioC_unstained' (DataFrame)


In [12]:
iRFP670_gate = 10000 
data_iRFP670gated = data[ data['iRFP670-A'] > iRFP670_gate]

mGL_gate=300
data_iRFP670gated['mGL_gate'] = np.where(data_iRFP670gated['mGL-A']>mGL_gate, '+', '-')

display( pd.unique(data_iRFP670gated['bioreplicate']))
display( data_iRFP670gated.head())

<ipython-input-12-2da7dde0a897>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_iRFP670gated['mGL_gate'] = np.where(data_iRFP670gated['mGL-A']>mGL_gate, '+', '-')


array(['20240304flow_exp20240301_p3_BioC', '20240309flow_exp20240306_p1',
       '20240309flow_exp20240306_p2',
       '20240316flow_exp20240313_p1_ZFtitr',
       '20240315flow_exp20240312_p3_ZFtitr'], dtype=object)

,reporter,ZF37,well,population,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,...,TagBFP-H,TagBFP-W,Time,bioreplicate,mCherry-A,log10 mGL-A,bioreplicate_reporter,cond,ZF37_str,mGL_gate
128,pTET021_203bp 0.1xCre,0.0,A10,Single Cells,361658,206641,110,314656,212225,117,...,71,0,1.51745,20240304flow_exp20240301_p3_BioC,122,1.113943,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,pTET021_203bp 0.1xCre.13 0.0\n16 ...,0.0,-
138,pTET021_203bp 0.1xCre,0.0,A10,Single Cells,392818,235527,115,432926,272793,118,...,54,0,1.59584,20240304flow_exp20240301_p3_BioC,7,2.152288,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,pTET021_203bp 0.1xCre.13 0.0\n16 ...,0.0,-
307,pTET021_203bp 0.1xCre,0.0,A10,Single Cells,368240,246940,108,179041,127093,113,...,62,0,3.13339,20240304flow_exp20240301_p3_BioC,13,1.755875,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,pTET021_203bp 0.1xCre.13 0.0\n16 ...,0.0,-
396,pTET021_203bp 0.1xCre,0.0,A10,Single Cells,439837,223571,117,430227,279300,126,...,80,0,4.03180,20240304flow_exp20240301_p3_BioC,152,1.869232,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,pTET021_203bp 0.1xCre.13 0.0\n16 ...,0.0,-
411,pTET021_203bp 0.1xCre,0.0,A10,Single Cells,400596,260131,110,195348,141697,116,...,37,0,4.20565,20240304flow_exp20240301_p3_BioC,12,1.681241,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,pTET021_203bp 0.1xCre.13 0.0\n16 ...,0.0,-


See Individual Replicate Analysis

In [13]:
# https://mitprod.sharepoint.com/:p:/s/GallowayLab/EQPVmV9kehhBgsdqt_Sk9O8BAfgiYRY1pIjHioyJ-aOPoA

## Analysis starts here

Histogram of titration

In [ ]:
# Palette
ZF37_palette = {  1: 'red', 
       0.5: 'orange',
        0.25:  'gold',
       0.125: 'lime',
      0.0625: 'teal',
        0.03125: 'blue',
        0.01563: 'plum' ,
        0.00781: 'darkviolet', 
        0: 'black'
}

# Joint Plots

In [ ]:
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

ycat = 'mCherry-A'; xcat = 'mGL-A'; hue = 'ZF37_str'; 

hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0", ])
       
palette = 'gist_heat'

for rept in repts: 
    data_now = data_iRFP670gated[ data_iRFP670gated['reporter']== rept]
    plottitle = rept + ' hist mChe'
    g = custom_density_plot(xcat=xcat,ycat=ycat,hue=hue,data=data_now,points = 10000, hue_order=hue_order,
                        savetitle='',plottitle=plottitle,palette=palette,
                        xlim=(0.5*10**0, 1*10**6),ylim = (0.5*10**0, 1*10**6),type=['contour'],alpha=0)
    #g.savefig(figure_folder+ plottitle + '.svg',dpi=300,bbox_inches='tight')


## Figure Joint Plot per Cre per ZF 

In [ ]:
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

ycat = 'mCherry-A'; xcat = 'mGL-A'; hue = 'ZF43_str'; 
ZF_list = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0", ])

palette = 'gist_heat'
hue = 'ZF37_str'
hue_order = ZF_list
vline=[200]
hline = 200

for rept in repts: 
    for i in range(len(ZF_list)):
        data_now = data_iRFP670gated[ data_iRFP670gated['reporter']== rept]
        data_now = data_now[ data_now['ZF37_str']== ZF_list[i]]
        plottitle = rept + ' ' + ZF_list[i]
        g = custom_density_plot(xcat=xcat,ycat=ycat,hue=hue,data=data_now,points = 10000, hue_order=hue_order,
                            savetitle='',plottitle='',palette=palette,
                            hline=hline, vline=vline,
                            legend=False,
                            xlim=(0.5*10**0, 1*10**6),ylim = (0.5*10**0, 1*10**6),type=['contour'],alpha=0)
        g.savefig(figure_folder+ plottitle + '.svg',dpi=300,bbox_inches='tight')


# Histograms 1D

In [ ]:
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

parameters = pd.array([ 'mCherry-A','mGL-A']); hue = 'ZF37_str'; 

hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0", ]); hueorder=hue_order;
       
palette = 'gist_heat'
xlim = [1e0,2e5]
xsize = 8
ysize = 1.8/1.2
fontsize = 16.5

#No gating on mGL 
for rept in repts: 
    data_now = data_iRFP670gated[ data_iRFP670gated['reporter']== rept]
    plottitle = rept + ' singlehis'
    #g = plt.figure(figsize=(xsize,ysize))
    g = custom_hist_plot( data_now, parameters, hue=hue, palette=palette, 
                         hueorder=hueorder, xsize=xsize, ysize=ysize, title=plottitle, 
                         alpha=0,
                         legendtitle='Condition', xlim=xlim, 
                         line=mGL_gate, 
                         fontsize=fontsize)
    g = g.get_figure()
    g.savefig(figure_folder+ plottitle + '.svg',dpi=300,bbox_inches='tight')

#All data together, no gating on mGL 
data_now = data_iRFP670gated[ data_iRFP670gated['reporter'].isin(repts)]
plottitle = 'all data singlehis'
g = custom_hist_plot( data_now, parameters, hue=hue, palette=palette, hueorder=hueorder, xsize=xsize, ysize=ysize, title=plottitle, 
                         alpha=0,fontsize=fontsize,
                         legendtitle='Condition', xlim=xlim)
g = g.get_figure()
g.savefig(figure_folder+ plottitle + '.svg',dpi=300,bbox_inches='tight')


# Calculate Summary Stats

In [14]:
df = data_iRFP670gated

by = ['reporter','ZF37','ZF37_str', 'bioreplicate', 'cond', 'bioreplicate_reporter'] #how to filter results, columns
x = ['mGL-A','iRFP670-A', 'mCherry-A'] #which parameter to calculate results on
stat = [sp.stats.gmean, np.median, np.std, np.mean] #statistics to calculate
s = calc_stats(df,by,x,stat)
#display(s)

#Make summary plots for the mGLgated on each side of the bimodality 
by = ['reporter','ZF37','ZF37_str', 'bioreplicate', 'cond', 'bioreplicate_reporter', 'mGL_gate'] #how to filter results, columns
s2 = calc_stats(df,by,x,stat)


In [15]:
# Save data 
s.to_excel('Fig2_S8_ZF37plasmidtitration.xlsx',index=False)
s2.to_excel('Fig2_S8_ZF37plasmidtitration_mGLgated.xlsx',index=False)

In [17]:
display(s)

,reporter,ZF37,ZF37_str,bioreplicate,cond,bioreplicate_reporter,mGL-A_gmean,mGL-A_median,mGL-A_std,mGL-A_mean,iRFP670-A_gmean,iRFP670-A_median,iRFP670-A_std,iRFP670-A_mean,mCherry-A_gmean,mCherry-A_median,mCherry-A_std,mCherry-A_mean,Fraction,Count
0,pTET021_203bp 0.1xCre,0.0,0.0,20240304flow_exp20240301_p3_BioC,pTET021_203bp 0.1xCre.13 0.0\n16 ...,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,65.022920,73.0,651.615103,130.220539,34783.275508,31047.0,49722.984881,50110.164983,39.494966,49.0,36.867438,54.203704,1.0,594
1,pTET021_203bp 0.1xCre,0.0,0.0,20240309flow_exp20240306_p1,pTET021_203bp 0.1xCre.13 0.0\n16 ...,20240309flow_exp20240306_p1_pTET021_203bp 0.1xCre,59.410203,68.0,653.770254,109.927971,34749.335616,31340.0,46214.401677,49396.276110,37.435091,46.0,36.312297,52.225290,1.0,2499
2,pTET021_203bp 0.1xCre,0.0,0.0,20240309flow_exp20240306_p2,pTET021_203bp 0.1xCre.13 0.0\n16 ...,20240309flow_exp20240306_p2_pTET021_203bp 0.1xCre,59.768403,69.0,319.510284,102.422126,45355.142621,46027.0,58479.769105,66037.527744,40.249401,50.0,39.939671,56.036128,1.0,5785
3,pTET021_203bp 0.1xCre,0.0,0.0,20240315flow_exp20240312_p3_ZFtitr,pTET021_203bp 0.1xCre.13 0.0\n16 ...,20240315flow_exp20240312_p3_ZFtitr_pTET021_203...,99.271289,101.0,325.871017,204.188645,41619.357269,38948.0,59266.793824,61888.630647,40.198697,49.0,38.143860,55.218559,1.0,1638
4,pTET021_203bp 0.1xCre,0.0,0.0,20240316flow_exp20240313_p1_ZFtitr,pTET021_203bp 0.1xCre.13 0.0\n16 ...,20240316flow_exp20240313_p1_ZFtitr_pTET021_203...,68.377090,76.0,189.054824,120.629692,39902.460817,37629.0,59093.420823,59480.985711,41.697839,51.0,47.347880,58.655607,1.0,4129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,pTET021_203bp 0xCre,1.0,1.0,20240304flow_exp20240301_p3_BioC,pTET021_203bp 0xCre.13 0.0\n16 ...,20240304flow_exp20240301_p3_BioC_pTET021_203bp...,273.213490,290.0,468.708104,407.357301,23486.619608,20183.0,28039.445576,30363.303097,2104.706647,2065.0,6917.717522,3967.311947,1.0,904
84,pTET021_203bp 0xCre,1.0,1.0,20240309flow_exp20240306_p1,pTET021_203bp 0xCre.13 0.0\n16 ...,20240309flow_exp20240306_p1_pTET021_203bp 0xCre,209.318394,229.0,347.975208,310.664187,22879.381398,20073.0,25083.757935,28755.505131,2158.333063,2035.0,6691.229855,3975.214726,1.0,3898
85,pTET021_203bp 0xCre,1.0,1.0,20240309flow_exp20240306_p2,pTET021_203bp 0xCre.13 0.0\n16 ...,20240309flow_exp20240306_p2_pTET021_203bp 0xCre,196.402210,209.0,364.621932,290.418706,24205.704989,21390.0,28843.043461,31131.977250,2015.237133,1864.0,8933.529814,4129.263296,1.0,9890
86,pTET021_203bp 0xCre,1.0,1.0,20240315flow_exp20240312_p3_ZFtitr,pTET021_203bp 0xCre.13 0.0\n16 ...,20240315flow_exp20240312_p3_ZFtitr_pTET021_203...,567.276147,569.0,1123.234546,844.105565,28046.547856,25251.0,35796.099829,37339.880666,2764.822134,2756.0,8489.582497,5052.522662,1.0,1743


## Summary Plots - Points are Bioreplicates

In [ ]:
#Make a list of the stats that I want to plot 
stat_list = ['mGL-A_gmean', 'mCherry-A_gmean', 'iRFP670-A_gmean' ]
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

#Plot the summary stats of mean and g.mean 
x = 'ZF37'; x_rot = 45
hue = 'reporter';  hueorder = repts; pairs = None
palette = 'muted'
order = None
pairs = [ [(1,'pTET021_203bp 0xCre'), (1,'pTET021_203bp 0.1xCre')], 
         [(0.5,'pTET021_203bp 0xCre'), (0.5,'pTET021_203bp 0.1xCre')], 
         [(0.25,'pTET021_203bp 0xCre'), (0.25,'pTET021_203bp 0.1xCre')], 
         [(0.125,'pTET021_203bp 0xCre'), (0.125,'pTET021_203bp 0.1xCre')], 
         [(0.0625,'pTET021_203bp 0xCre'), (0.0625,'pTET021_203bp 0.1xCre')],
         [(0.03125,'pTET021_203bp 0xCre'), (0.03125,'pTET021_203bp 0.1xCre')],
         [(0.125,'pTET021_203bp 0.1xCre'), (1,'pTET021_203bp 0.1xCre')], 
        [(0.125,'pTET021_203bp 0xCre'), (1,'pTET021_203bp 0xCre')]
         ]

yscales = pd.array(['log', 'linear'])
ylim = [1e1, 0.3e4]


data_now = s[ s['reporter'].isin(repts)]
for stat in stat_list: 
    y = stat
    for yscale_now in yscales:
            yscale = yscale_now
            plottitle = 'Cre_add ' + stat + yscale_now
            g = summary_plot( x, y, hue, data=data_now, dodge=True, palette = palette, order=order, 
                                hue_order = hueorder, plottitle=plottitle,yscale=yscale,x_rot = x_rot, 
                                ylim = ylim, pairs = pairs)
            g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')


Summary plot by bioreplicate

In [ ]:
df = data_iRFP670gated

by = ['reporter','ZF37','ZF37_str', 'bioreplicate', 'cond', 'bioreplicate_reporter','well'] #how to filter results, columns
x = ['mGL-A','iRFP670-A', 'mCherry-A'] #which parameter to calculate results on
stat = [sp.stats.gmean, np.median, np.std, np.mean] #statistics to calculate
s2 = calc_stats(df,by,x,stat)


#Make a list of the stats that I want to plot 
stat_list = ['mGL-A_gmean']
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

#Plot the summary stats of mean and g.mean 
x = 'ZF37'; x_rot = 45
hue = 'reporter';  hueorder = repts; pairs = None
palette = 'muted'
order = None
pairs = [ ]

yscales = pd.array(['log'])
ylim = [1e1, 1e4]

bioreps = pd.unique( s['bioreplicate'])

for biorep in bioreps:
    s_now = s2[ s2['bioreplicate']==biorep]
    data_now = s_now[ s_now['reporter'].isin(repts)]
    for stat in stat_list: 
        y = stat
        for yscale_now in yscales:
                yscale = yscale_now
                plottitle = biorep
                g = summary_plot( x, y, hue, data=data_now, dodge=True, palette = palette, order=order, 
                                    hue_order = hueorder, plottitle=plottitle,yscale=yscale,x_rot = x_rot, 
                                    ylim = ylim, pairs = pairs, legend=False)
                #g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')


 ## Normalize by ZF at 0.125x and 1x (no Cree)

In [16]:

#For each bioreplicate, get the value at ZF43 at 0.125 and append it to a dictionary

#start dictionary with values that dont matter
map_mGLnorm = { 'test': 'test', 'test2': 'test2'} #for normalizing mGL
map_mChnorm = {'test':'test'}
map_mGLnorm2 = {'test':'test'}
map_mChnorm2 = {'test':'test'}
#only get the relevant part of the s dataframe
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])
s_subset = s[ s['reporter'].isin(repts)]
s2_subset = s2[ s2['reporter'].isin(repts)]; s2_subset= s2_subset[ s2_subset['mGL_gate']=='+']
#get the list of the bioreplicates 
biorep_repts = pd.unique( s_subset['bioreplicate'])


for rep in biorep_repts: # for each biorep
    
    #All data
    s_now = s_subset[ s_subset['bioreplicate'] == rep] # go to that specific biorep 
    s_now = s_now[ s_now['reporter'] == 'pTET021_203bp 0xCre'] 
    s_now = s_now[ s_now['ZF37'] == 0.125] #select that specific ZF value
    #print(s_now['mGL-A_gmean'], type(s_now['mGL-A_gmean']))
    map_mGLnorm[rep] = s_now.iloc[0]['mGL-A_gmean']
    map_mChnorm[rep]= s_now.iloc[0]['mCherry-A_gmean']

    # s2_subset -- mGL positive only 
    s_now = s2_subset[ s2_subset['bioreplicate'] == rep] # go to that specific biorep 
    s_now = s_now[ s_now['reporter'] == 'pTET021_203bp 0xCre'] 
    s_now = s_now[ s_now['ZF37'] == 0.125] #select that specific ZF value
    #print(s_now['mGL-A_gmean'], type(s_now['mGL-A_gmean']))
    map_mGLnorm2[rep] = s_now.iloc[0]['mGL-A_gmean']
    map_mChnorm2[rep]= s_now.iloc[0]['mCherry-A_gmean']

#Map the values to the other column 
#All data
s_subset['ZF37_0.125_mGL'] = s_subset['bioreplicate'].map(map_mGLnorm).astype(float)
s_subset['mGL-A_gmean norm ZF37_0.125'] = (s_subset['mGL-A_gmean']) / (s_subset['ZF37_0.125_mGL'])
s_subset['ZF37_0.125_mCherry'] = s_subset['bioreplicate'].map(map_mChnorm).astype(float)
s_subset['mCherry-A_gmean norm ZF37_0.125'] = (s_subset['mCherry-A_gmean']) / (s_subset['ZF37_0.125_mCherry'])
# mGL positive only 
s2_subset['ZF37_0.125_mGL'] = s2_subset['bioreplicate'].map(map_mGLnorm2).astype(float)
s2_subset['mGL-A_gmean norm ZF37_0.125'] = (s2_subset['mGL-A_gmean']) / (s2_subset['ZF37_0.125_mGL'])
s2_subset['ZF37_0.125_mCherry'] = s2_subset['bioreplicate'].map(map_mChnorm2).astype(float)
s2_subset['mCherry-A_gmean norm ZF37_0.125'] = (s2_subset['mCherry-A_gmean']) / (s2_subset['ZF37_0.125_mCherry'])
# Make means of each s_subset and s2_subset 
by =['reporter','ZF37','ZF37_str'];  
x = ['mGL-A_gmean','iRFP670-A_gmean', 'mCherry-A_gmean', 'Fraction', 'mCherry-A_gmean norm ZF37_0.125', 'mGL-A_gmean norm ZF37_0.125'] 
stat = [np.std, np.mean] #statistics to calculate
df = s_subset; s_subset_means = calc_stats(df,by,x,stat)
df = s2_subset; s2_subset_means = calc_stats(df,by,x,stat)
display(s_subset_means)
# #--------------
# # Normalize by 1X 
# #start dictionary with values that dont matter
# map_ZF43norm2 = { 'test': 'test', 'test2': 'test2'}
# for rep in biorep_repts: # for each biorep
#     s_now = s_subset[ s_subset['bioreplicate'] == rep] # go to that specific biorep 
#     s_now = s_now[ s_now['reporter'] == 'pTET021_203bp 0xCre'] 
#     s_now = s_now[ s_now['ZF37'] == 1] #select that specific ZF43 value
#     #print(s_now['mGL-A_gmean'], type(s_now['mGL-A_gmean']))
#     map_ZF43norm2[rep] = s_now.iloc[0]['mGL-A_gmean'] 
# #Map the values to the other column 
# s_subset['ZF37_1'] = s_subset['bioreplicate'].map(map_ZF43norm2).astype(float)
# s_subset['mGL-A_gmean norm ZF37_1'] = (s_subset['mGL-A_gmean']) / (s_subset['ZF37_1'])
# s_subset['mGL-A_gmean norm ZF37_1 LOG'] = np.log10(s_subset['mGL-A_gmean']) / np.log10(s_subset['ZF37_1'])

,reporter,ZF37,ZF37_str,mGL-A_gmean_std,mGL-A_gmean_mean,iRFP670-A_gmean_std,iRFP670-A_gmean_mean,mCherry-A_gmean_std,mCherry-A_gmean_mean,Fraction_std,Fraction_mean,mCherry-A_gmean norm ZF37_0.125_std,mCherry-A_gmean norm ZF37_0.125_mean,mGL-A_gmean norm ZF37_0.125_std,mGL-A_gmean norm ZF37_0.125_mean,Fraction,Count
0,pTET021_203bp 0.1xCre,0.00000,0.0,16.585826,70.369981,4569.322717,39281.914366,1.553013,39.815199,0.0,1.0,0.035515,0.120666,0.055885,0.209822,1.0,5
1,pTET021_203bp 0.1xCre,0.00781,0.00781,719.562416,878.456275,5932.088965,41495.033388,16.298271,86.288736,0.0,1.0,0.037872,0.252255,0.862390,2.242875,1.0,5
2,pTET021_203bp 0.1xCre,0.01563,0.01563,1403.983213,1647.229494,5217.026592,39690.796992,32.453790,115.128890,0.0,1.0,0.078337,0.335632,1.952244,4.081674,1.0,5
3,pTET021_203bp 0.1xCre,0.03125,0.03125,2153.466508,2804.369700,3688.206381,39188.430771,65.526952,202.954653,0.0,1.0,0.065624,0.575844,2.216458,7.036871,1.0,5
4,pTET021_203bp 0.1xCre,0.06250,0.0625,2067.667303,3562.686017,1401.461500,36088.594924,56.761883,266.214171,0.0,1.0,0.087650,0.776823,1.639398,8.497676,1.0,4
5,pTET021_203bp 0.1xCre,0.12500,0.125,3360.691294,4443.326883,3476.310455,34390.844543,149.809197,485.432945,0.0,1.0,0.132017,1.379221,3.394109,10.868759,1.0,5
6,pTET021_203bp 0.1xCre,0.25000,0.25,3833.517953,4815.858323,5002.819734,31932.644820,290.254428,797.660266,0.0,1.0,0.453595,2.268246,4.575019,11.496851,1.0,5
7,pTET021_203bp 0.1xCre,0.50000,0.5,3982.616149,4846.323932,4147.359806,32278.980398,416.267030,1555.702053,0.0,1.0,0.306057,4.454212,4.803024,11.462611,1.0,5
8,pTET021_203bp 0.1xCre,1.00000,1.0,2636.622747,3567.929464,3110.288907,29821.170262,532.630443,2870.641246,0.0,1.0,0.791014,8.343676,3.039189,8.733708,1.0,5
9,pTET021_203bp 0xCre,0.00000,0.0,2.317249,39.738164,3054.645035,32260.565082,1.617182,36.971452,0.0,1.0,0.031754,0.111851,0.045166,0.124249,1.0,5


## Plot Just mGL SUmmary Plot (not Scatter plot)

In [ ]:
#Make a list of the stats that I want to plot 
stat_list = ['mGL-A_gmean norm ZF37_0.125']
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

#Plot the summary stats of mean and g.mean 
x = 'ZF37'; x_rot = 45
hue = 'reporter';  hueorder = repts; pairs = None
palette = 'muted'
order = None
pairs = [ [(1,'pTET021_203bp 0xCre'), (1,'pTET021_203bp 0.1xCre')], 
         [(0.5,'pTET021_203bp 0xCre'), (0.5,'pTET021_203bp 0.1xCre')], 
         [(0.25,'pTET021_203bp 0xCre'), (0.25,'pTET021_203bp 0.1xCre')], 
         [(0.125,'pTET021_203bp 0xCre'), (0.125,'pTET021_203bp 0.1xCre')], 
         [(0.0625,'pTET021_203bp 0xCre'), (0.0625,'pTET021_203bp 0.1xCre')],
         [(0.03125,'pTET021_203bp 0xCre'), (0.03125,'pTET021_203bp 0.1xCre')],
         [(0.125,'pTET021_203bp 0.1xCre'), (1,'pTET021_203bp 0.1xCre')], 
        [(0.125,'pTET021_203bp 0xCre'), (1,'pTET021_203bp 0xCre')]
         ]

yscales = pd.array(['linear'])
ylim = [0,1.25]

data_now = s_subset[ s_subset['reporter'].isin(repts)]
for stat in stat_list: 
    y = stat
    for yscale_now in yscales:
            yscale = yscale_now
            plottitle = stat
            g = summary_plot( x, y, hue, data=data_now, dodge=True, palette = palette, order=order, 
                                hue_order = hueorder, plottitle=plottitle,yscale=yscale,x_rot = x_rot, 
                                ylim = ylim,pairs = pairs, legend=False)
            #g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

## Scatter Plots unormalized

In [ ]:
#only get the relevant part of the s dataframe
df = data_iRFP670gated

by = ['reporter','ZF37','ZF37_str', 'bioreplicate', 'cond', 'bioreplicate_reporter'] #how to filter results, columns
x = ['mGL-A','iRFP670-A', 'mCherry-A','TagBFP-A'] #which parameter to calculate results on
stat = [sp.stats.gmean, np.median, np.std, np.mean] #statistics to calculate
s3 = calc_stats(df,by,x,stat)

repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])
data_now = s3[ s3['reporter'].isin(repts)]
ZF37_palette = 'gist_heat'
y= 'mGL-A_gmean'
x = 'mCherry-A_gmean'
hue = 'ZF37_str'
palette = ZF37_palette
style = 'reporter'
ylim = [1e1, 1e4]
xlim = ylim
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])

plottitle = 'Scatterplot all'
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 50, legend=False)
#g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

# #add in data from promoters 
# data_now = s_promoters37
# hue = 'ZF37'
# palette = 'Blues'
# hue_order = None
# g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
#                  plottitle=plottitle, style=style, hue_order=hue_order,
#                  ylim=ylim, xlim=xlim, s= 50, legend=False)

In [ ]:
display()

## Scatter Plot Normalized mGL-0.125x, mCherry-unormalized

In [ ]:
#only get the relevant part of the s dataframe
data_now = s_subset[ s_subset['ZF37_str']!= '0.0']
x= 'mCherry-A_gmean'
hue = 'ZF37_str'
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
style = 'reporter'
xlim = [1e1, 1e4]
ylim = [0.1, 100]
yscale = 'log'
xlabel = 'mCherry-A gmean norm.'
ylabel = 'mGL-A gmean norm.'
xsize = 3
ysize = 3


stat_list = ['mGL-A_gmean norm ZF37_0.125']
for stat in stat_list:
  y = stat
  plottitle = 'Scatterplot  0.125x'
  plt.figure(figsize=(xsize,ysize))
  g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                  plottitle=plottitle, style=style, hue_order=hue_order,
                  ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, legend=False)
  #g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')


# plottitle = 'Scatterplot  0.125x mGLpos'
# data_now = s2_subset[ s2_subset['ZF37_str']!= '0.0']
# plt.figure(figsize=(xsize,ysize))
# h = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
#                  plottitle=plottitle, style=style, hue_order=hue_order,
#                  ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel)
# h.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')
# #both 

## Scatterplot both mGL and mCherry 0.125x norm

In [ ]:
# both normalized
#only get the relevant part of the s dataframe
markers = ['o','s']
data_now = s_subset
x= 'mCherry-A_gmean norm ZF37_0.125'
hue = 'ZF37_str'
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
style = 'reporter'
xlim = [0.07,15]
ylim = [0.07,2e1]
yscale = 'log'
stat_list = ['mGL-A_gmean norm ZF37_0.125']
xlabel = 'mCherry-A gmean norm.'
ylabel = 'mGL-A gmean norm.'
fontsize = 15; 
xsize = 3; ysize=xsize

for stat in stat_list:
  y = stat
plottitle = 'Scatterplot both norm 0.125x'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False)
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

## Means version 
data_now = s_subset_means

x = 'mCherry-A_gmean norm ZF37_0.125_mean'
y = 'mGL-A_gmean norm ZF37_0.125_mean'
x_std = 'mCherry-A_gmean norm ZF37_0.125_std'
y_std = 'mGL-A_gmean norm ZF37_0.125_std'
plottitle = 'Scatterplot both norm 0.125x MEAN'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 100, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False, std_x=x_std, std_y=y_std)
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

ylim = [0.5,2e1]
plottitle = 'Scatterplot both norm 0.125x mGLpos'
xlim = [0.07,15]
ylim = [0.07,2e1]
xlabel = 'Input ZFa gmean\n(norm., mCherry)'
ylabel = 'Output gmean\n(norm., mGL)'
data_now = s2_subset_means[ s2_subset_means['ZF37_str']!= '0.0']
plt.figure(figsize=(xsize,ysize))
h = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order, markers=markers,
                 ylim=ylim, xlim=xlim, s= 100, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize,legend=False, std_x=x_std, std_y=y_std)
h.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

plottitle = 'Scatterplot norm 0.125x mGLpos vs Fraction'
x = 'Fraction_mean'
x_std = ''
data_now = s2_subset_means[ s2_subset_means['ZF37_str']!= '0.0']
#data_now = data_now[ data_now['reporter']=='pTET021_203bp 0.1xCre']
plt.figure(figsize=(xsize,ysize))
h = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 100, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize,legend=False, std_y=y_std, 
                 xscale='linear')
h.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

# #Overaly plot of the different promoters 
# plottitle = 'Scatterplot both norm0.125x mGL prom'
# data_now = s2_subset[ s2_subset['ZF37_str']!= '0.0']
# palette = ['black','gray']; hue= 'reporter'; hue_order = None; 
# plt.figure(figsize=(xsize,ysize))
# k = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
#                  plottitle=plottitle, style=style, hue_order=hue_order,
#                  ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
#                  fontsize=fontsize, labelsize=fontsize)
# hue = 'ZF37'; hue_order = pd.array( [ 'pTET112 hPGK.d','pTET111 duplex UBC.d','pTET EF1a 1x','pTET113 CMV'])
# data_now = s_promoters37_normEF1a[s_promoters37_normEF1a['ZF37'].isin(hue_order)]
# palette = 'gist_heat'
# x = 'mCherry-A_gmean norm ZF37_EF1a'; y = 'mGL-A_gmean norm ZF37_EF1a'
# k = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
#                  plottitle=plottitle, style=style, hue_order=hue_order,
#                  ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
#                  fontsize=fontsize, labelsize=fontsize)
# k.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

In [ ]:
sns.plotting_context(context='paper')['font.size']

OVERLAYING MODELING VALUES

In [ ]:
# single curve per condition based on MATLAB generated parameters

data_now = s_subset

# identify leaky values
leaky_exp_noCre = data_now.loc[data_now['reporter'] == 'pTET021_203bp 0xCre', 'mCherry-A_gmean norm ZF37_0.125'].min()
leaky_exp_Cre = data_now.loc[data_now['reporter'] == 'pTET021_203bp 0.1xCre', 'mCherry-A_gmean norm ZF37_0.125'].min()

# use to adjust the datapoints if using normal axes
# data_now.loc[data_now['reporter'] == 'pTET021_203bp 0xCre', 'mCherry-A_gmean norm ZF37_0.125'] -= leaky_exp_noCre
# data_now.loc[data_now['reporter'] == 'pTET021_203bp 0.1xCre', 'mCherry-A_gmean norm ZF37_0.125'] -= leaky_exp_Cre

y= 'mGL-A_gmean norm ZF37_0.125'
x= 'mCherry-A_gmean norm ZF37_0.125'
hue = 'ZF37_str'
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
style = 'reporter'
xlim = [0.04,15]
ylim = [0.04,200]
yscale = 'log'
xscale = 'log'
stat_list = ['mGL-A_gmean norm ZF37_0.125']
xlabel = 'mCherry-A gmean norm.'
ylabel = 'mGL-A gmean norm.'
fontsize = 15; 
xsize = 4; ysize=xsize

# Define the function
def overlay_curve(x, k_cat_prime, K_M, n, alpha_prime):
    return k_cat_prime * (x ** n) / (K_M + (x ** n)) + alpha_prime

# Parameters for the curve, taken from MATLAB
n = 1

k_cat_prime_noCre = 0.9834126
K_M_noCre = 0.2114736
alpha_prime_noCre = 0.1397989

k_cat_prime_Cre = 15.3733
K_M_Cre = 0.4870606
alpha_prime_Cre = 0.220971


# Generate x values for plotting the curve
x_values = np.linspace(0, max(data_now['mCherry-A_gmean norm ZF37_0.125']), 1000)

# Calculate corresponding y values using the overlay_curve function
# add leaky exp to shift the line 
y_values_noCre = overlay_curve(x_values, k_cat_prime_noCre, K_M_noCre, n, alpha_prime_noCre)
y_values_Cre = overlay_curve(x_values, k_cat_prime_Cre, K_M_Cre, n, alpha_prime_Cre)

x_values_noCre = x_values + leaky_exp_noCre
x_values_Cre = x_values + leaky_exp_Cre

## Plot on the data for all of the bioreps 

#Plot Scatterplot 
plottitle = 'Scat norm 0.125x +model'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, yscale=yscale, xscale=xscale, xlim=xlim, ylim=ylim,
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 s= 50, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False)
# Plot the overlay curve on the existing plot
plt.plot(x_values_noCre, y_values_noCre, color='darksalmon')
plt.plot(x_values_Cre, y_values_Cre, color='red')
# Show the plot
plt.show()
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

In [ ]:
# importing 95% CI calculated in MATLAB to plot -- need to troubleshoot
filepath_ci_noCre = './datafiles/ci_data_noCre_ZF37.csv'
filepath_ci_Cre = './datafiles/ci_data_Cre_ZF37.csv'

ci_data_noCre = pd.read_csv(filepath_ci_noCre, header=None, names=['mCherry-A', 'Lower Percentile', 'Upper Percentile'])
ci_data_Cre = pd.read_csv(filepath_ci_Cre, header=None, names=['mCherry-A', 'Lower Percentile', 'Upper Percentile'])

# transformed the data for modeling purposes -- must transform back to overlay on raw data

leaky_exp_noCre = 0.0980; leaky_exp_Cre = 0.1039

ci_data_noCre['mCherry-A'] = ci_data_noCre['mCherry-A'] + leaky_exp_noCre
ci_data_Cre['mCherry-A'] = ci_data_Cre['mCherry-A'] + leaky_exp_Cre

y= 'mGL-A_gmean norm ZF37_0.125'
x= 'mCherry-A_gmean norm ZF37_0.125'
hue = 'ZF37_str'
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
style = 'reporter'
xlim = [0.04,15]
ylim = [0.04,200]
yscale = 'log'
xscale = 'log'
stat_list = ['mGL-A_gmean norm ZF37_0.125']
xlabel = 'mCherry-A gmean norm.'
ylabel = 'mGL-A gmean norm.'
fontsize = 15; 
xsize = 4; ysize=xsize

plottitle = 'Scat norm 0.125x +model'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, yscale='log', xscale='log', xlim=xlim, ylim=ylim,
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 s= 50, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False)



# Overlay the confidence intervals on the scatter plot
plt.plot(ci_data_noCre['mCherry-A'], ci_data_noCre['Lower Percentile'], color='blue', linestyle='--', label='No Cre (2.5% Percentile)')
plt.plot(ci_data_noCre['mCherry-A'], ci_data_noCre['Upper Percentile'], color='blue', linestyle='--', label='No Cre (97.5% Percentile)')
plt.plot(ci_data_Cre['mCherry-A'], ci_data_Cre['Lower Percentile'], color='green', linestyle='--', label='Cre (2.5% Percentile)')
plt.plot(ci_data_Cre['mCherry-A'], ci_data_Cre['Upper Percentile'], color='green', linestyle='--', label='Cre (97.5% Percentile)')

# Add legend and title
#plt.legend()
plt.title('Scatter Plot with 95% Confidence Intervals')

plt.xticks([])
plt.yticks([])

# Show the plot
plt.show()

# Import parameters for bootstrapped model fitting

In [ ]:
# generating confidence intervals in python after using MATLAB generated parameters (kcat and KM)
# import matlab generated values

filepath_kcat_noCre = './datafiles/kcat_noCre_ZF37.csv'
filepath_kcat_Cre = './datafiles/kcat_Cre_ZF37.csv'
filepath_KM_noCre = './datafiles/KM_noCre_ZF37.csv'
filepath_KM_Cre = './datafiles/KM_Cre_ZF37.csv'

kcat_noCre = pd.read_csv(filepath_kcat_noCre, header=None, names=['noCre'])
kcat_Cre = pd.read_csv(filepath_kcat_Cre, header=None, names=['Cre'])
KM_noCre = pd.read_csv(filepath_KM_noCre, header=None, names=['noCre'])
KM_Cre = pd.read_csv(filepath_KM_Cre, header=None, names=['Cre'])


In [ ]:
# format kcat and KM into df
kcat = pd.concat([kcat_noCre, kcat_Cre], axis=1)
km = pd.concat([KM_noCre, KM_Cre], axis=1)

# display(kcat) # for troubleshooting
# display(km)

In [ ]:
# using defined function to calculate corresponding curve for each bootstrapped parameter set

data_now = s_subset

# previously defined function
def overlay_curve(x, k_cat_prime, K_M, n, alpha_prime):
    return k_cat_prime * (x ** n) / (K_M + (x ** n)) + alpha_prime

# Parameters for the curve, taken from MATLAB
n = 1
alpha_prime_noCre = 0.1397989
alpha_prime_Cre = 0.220971

# identify leaky values
leaky_exp_noCre = data_now.loc[data_now['reporter'] == 'pTET021_203bp 0xCre', 'mCherry-A_gmean norm ZF37_0.125'].min()
leaky_exp_Cre = data_now.loc[data_now['reporter'] == 'pTET021_203bp 0.1xCre', 'mCherry-A_gmean norm ZF37_0.125'].min()

x_values = np.linspace(0, 40, 3000)

# initialize
y_values_noCre = pd.DataFrame()
y_values_Cre = pd.DataFrame()

for index, row in kcat.iterrows():
    # Get parameters for the current row
    k_cat_prime = row['noCre']
    k_M = km.loc[index, 'noCre']
    
    # Calculate corresponding y values using the overlay_curve function
    y_values = overlay_curve(x_values, k_cat_prime, k_M, n, alpha_prime_noCre)
    
    # Store y_values in the DataFrame for noCre
    y_values_noCre[index] = y_values

y_values_noCre = y_values_noCre.transpose()

# Run the loop for 'Cre'
for index, row in kcat.iterrows():
    # Get parameters for the current row
    k_cat_prime = row['Cre']
    k_M = km.loc[index, 'Cre']
    
    # Calculate corresponding y values using the overlay_curve function
    y_values = overlay_curve(x_values, k_cat_prime, k_M, n, alpha_prime_Cre)
    
    # Store y_values in the DataFrame for Cre
    y_values_Cre[index] = y_values

y_values_Cre = y_values_Cre.transpose()

# display(y_values_noCre) # for troubleshooting

In [ ]:
# using defined function to calculate corresponding curve for each bootstrapped parameter set

# calculate mean values for each parameter for center line
mean_kcat_noCre = kcat['noCre'].mean()
mean_kcat_Cre = kcat['Cre'].mean()
mean_km_noCre = km['noCre'].mean()
mean_km_Cre = km['Cre'].mean()

y_values_noCre_mean = overlay_curve(x_values, mean_kcat_noCre, mean_km_noCre, n, alpha_prime_noCre)
y_values_Cre_mean = overlay_curve(x_values, mean_kcat_Cre, mean_km_Cre, n, alpha_prime_Cre)

display(y_values_Cre_mean)

## Scatterplot with 95% CI Correct

In [ ]:
markers = ['o', 's']

# sort the resulting curves in ascending order per x value
y_values_noCre_sorted = y_values_noCre.apply(lambda col: col.sort_values().values)
y_values_Cre_sorted = y_values_Cre.apply(lambda col: col.sort_values().values)

# keep only the 2.5% and 97.5% percentile (95% CI)
y_values_noCre_lower = y_values_noCre_sorted.iloc[249]
y_values_Cre_lower = y_values_Cre_sorted.iloc[249]
y_values_noCre_higher = y_values_noCre_sorted.iloc[9749]
y_values_Cre_higher = y_values_Cre_sorted.iloc[9749]

x_values_noCre = x_values + leaky_exp_noCre
x_values_Cre = x_values + leaky_exp_Cre

# plot the original scatterplot
#Plot Scatterplot 
data_now = s_subset_means
style = 'reporter'; hue=hue = 'ZF37_str'; hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
x = 'mCherry-A_gmean norm ZF37_0.125_mean'
y = 'mGL-A_gmean norm ZF37_0.125_mean'
xlabel = 'Input ZFa gmean\n(norm., mCherry)'
ylabel = 'Output gmean\n(norm., mGL)'
xsize = 3
ysize = 3
xlim = [0.07,15]
ylim = [0.07,2e1]

x_std = 'mCherry-A_gmean norm ZF37_0.125_std'
y_std = 'mGL-A_gmean norm ZF37_0.125_std'
plottitle = 'Scatterplot both norm 0.125x MEAN + 95CI'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, markers=markers,
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 100, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False, std_x=x_std, std_y=y_std)

# # Plot the overlay curve on the existing plot
# plt.plot(x_values_noCre, y_values_noCre_lower, color='darksalmon')
# plt.plot(x_values_noCre, y_values_noCre_higher, color='darksalmon')
# plt.plot(x_values_Cre, y_values_Cre_lower, color='red')
# plt.plot(x_values_Cre, y_values_Cre_higher, color='red')

# Plot the lightly shaded area between the curves
plt.fill_between(x_values_noCre, y_values_noCre_lower, y_values_noCre_higher, color='grey', alpha=0.2)
plt.fill_between(x_values_Cre, y_values_Cre_lower, y_values_Cre_higher, color='grey', alpha=0.2)

g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

# second iteration
plottitle = 'Scatterplot both norm 0.125x MEAN + 95CI + mean curve'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, markers=markers,
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 100, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False, std_x=x_std, std_y=y_std)

plt.plot(x_values_noCre, y_values_noCre_mean, color='grey', linewidth=0.5)
plt.plot(x_values_Cre, y_values_Cre_mean, color='grey', linewidth=0.5)

plt.fill_between(x_values_noCre, y_values_noCre_lower, y_values_noCre_higher, color='grey', alpha=0.2)
plt.fill_between(x_values_Cre, y_values_Cre_lower, y_values_Cre_higher, color='grey', alpha=0.2)
# Show the plot
plt.show()
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

# with promoters
plottitle = 'Scatterplot promoters MEAN + 95CI + mean curve'
plt.figure(figsize=(xsize,ysize))
## NEED TO ADD IN DATA FOR PROMOTERS HERE, wasn't saved last time  
data_now = s_promoters37_normEF1a_means
hue_order = pd.array( ['no ZF', 'pTET112 hPGK.d','pTET111 duplex UBC.d','pTET EF1a 1x','pTET113 CMV']) #take out 'pTET EF1a 0.125x' for plot
data_now = data_now[data_now['ZF37'].isin(hue_order)]
x = 'mCherry-A_gmean norm ZF37_EF1a_mean'
y = 'mGL-A_gmean norm ZF37_EF1a_mean'
x_std = 'mCherry-A_gmean norm ZF37_EF1a_std'
y_std = 'mGL-A_gmean norm ZF37_EF1a_std'
hue = 'ZF37'
xlim = [0.07,40]
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, markers=markers,
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 100, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize, legend=False, std_x=x_std, std_y=y_std)

plt.plot(x_values_noCre, y_values_noCre_mean, color='grey', linewidth=0.5)
plt.plot(x_values_Cre, y_values_Cre_mean, color='grey', linewidth=0.5)

plt.fill_between(x_values_noCre, y_values_noCre_lower, y_values_noCre_higher, color='grey', alpha=0.2)
plt.fill_between(x_values_Cre, y_values_Cre_lower, y_values_Cre_higher, color='grey', alpha=0.2)
# Show the plot
plt.show()
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')


In [ ]:
display(s_promoters37_normEF1a_means)

## Parameter plots by themselves

In [ ]:
# generating summary plots with 95% CI error bars
mean_values = kcat.mean()
percentiles = kcat.quantile([0.025, 0.975])

# Calculate error bars
lower_error = mean_values.values - percentiles.loc[0.025].values
upper_error = percentiles.loc[0.975].values - mean_values.values

# Plot
plottitle='95_CI_kcat_ZF37'

plt.figure(figsize=(1, 1))

plt.errorbar(x=mean_values.index, y=mean_values.values, 
             yerr=[lower_error, upper_error], 
             fmt='o', capsize=5, color='black')

plt.title('k_cat', fontsize=15)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.ylim(0,21)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.show()

g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

In [ ]:
mean_values = km.mean()
percentiles = km.quantile([0.025, 0.975])

# Plot
plottitle='95_CI_KM_ZF37'

plt.figure(figsize=(1, 1))

plt.errorbar(x=mean_values.index, y=mean_values.values, 
             yerr=np.abs(mean_values.values - percentiles.loc[0.025].values), 
             fmt='o', capsize=5, color='black')

plt.title('K_M', fontsize=15)

plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

plt.ylim(0,0.8)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.show()

g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

## Scatter plot normalized to 1

In [ ]:
#only get the relevant part of the s dataframe
data_now = s_subset
x= 'mCherry-A_gmean'
hue = 'ZF37_str'
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
style = 'reporter'
xlim = [1e1, 1e4]
ylim = [0.1, 20]
yscale = 'log'
stat_list = ['mGL-A_gmean norm ZF37_1']
for stat in stat_list:
  y = stat
plottitle = 'Scatterplot normalized 1x'
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 50)
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

## Normalize by iRFP670

In [ ]:
s['mGL-A_gmean norm iRFP670'] = s['mGL-A_gmean'] / s['iRFP670-A_gmean']

#Make a list of the stats that I want to plot 
stat_list = ['mGL-A_gmean norm iRFP670' ]
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

#Plot the summary stats of mean and g.mean 
x = 'ZF43'; x_rot = 45
hue = 'reporter';  hueorder = repts; pairs = None
palette = 'muted'
order = None

yscales = pd.array(['log', 'linear'])

data_now = s[ s['reporter'].isin(repts)]
for stat in stat_list: 
    y = stat
    for yscale_now in yscales:
            yscale = yscale_now
            plottitle = 'Cre_add ' + stat
            g = summary_plot( x, y, hue, data=data_now, dodge=True, palette = palette, order=order, 
                                hue_order = hueorder, plottitle=plottitle,yscale=yscale,x_rot = x_rot, 
                                #ylim = ylim
                                )
            g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

## Summary PLots - separated -Cre, +Cre

In [ ]:
#Make a list of the stats that I want to plot 
stat_list = ['mGL-A_gmean', 'TagBFP-A_gmean' ]
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])

#Plot the summary stats of mean and g.mean 
x = 'ZF43'; x_rot = 45
hue = None;  hueorder = None; pairs = None
palette = 'gist_heat'
order = None

yscales = pd.array(['log', 'linear'])

for rept in repts:
    data_now = s[ s['reporter']==rept]
    for stat in stat_list: 
        y = stat
        for yscale_now in yscales:
                yscale = yscale_now
                plottitle = rept + ' ' + stat
                g = summary_plot( x, y, hue, data=data_now, dodge=True, palette = palette, order=order, 
                                hue_order = hueorder, plottitle=plottitle,yscale=yscale,x_rot = x_rot)
                g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

Normalize and Plot by 0.5X

In [ ]:
#For each bioreplicate, get the value at ZF43 at 0.125 and append it to a dictionary

#start dictionary with values that dont matter
map_mGLnorm = { 'test': 'test', 'test2': 'test2'} #for normalizing mGL
map_mChnorm = {'test':'test'}
map_mGLnorm2 = {'test':'test'}
map_mChnorm2 = {'test':'test'}
#only get the relevant part of the s dataframe
repts = pd.array( [ 'pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])
s_subset = s[ s['reporter'].isin(repts)]
s2_subset = s2[ s2['reporter'].isin(repts)]; s2_subset= s2_subset[ s2_subset['mGL_gate']=='+']
#get the list of the bioreplicates 
biorep_repts = pd.unique( s_subset['bioreplicate'])


for rep in biorep_repts: # for each biorep
    
    #All data
    s_now = s_subset[ s_subset['bioreplicate'] == rep] # go to that specific biorep 
    s_now = s_now[ s_now['reporter'] == 'pTET021_203bp 0xCre'] 
    s_now = s_now[ s_now['ZF37'] == 0.5] #select that specific ZF value
    #print(s_now['mGL-A_gmean'], type(s_now['mGL-A_gmean']))
    map_mGLnorm[rep] = s_now.iloc[0]['mGL-A_gmean']
    map_mChnorm[rep]= s_now.iloc[0]['mCherry-A_gmean']

    # s2_subset -- mGL positive only 
    s_now = s2_subset[ s2_subset['bioreplicate'] == rep] # go to that specific biorep 
    s_now = s_now[ s_now['reporter'] == 'pTET021_203bp 0xCre'] 
    s_now = s_now[ s_now['ZF37'] == 0.5] #select that specific ZF value
    #print(s_now['mGL-A_gmean'], type(s_now['mGL-A_gmean']))
    map_mGLnorm2[rep] = s_now.iloc[0]['mGL-A_gmean']
    map_mChnorm2[rep]= s_now.iloc[0]['mCherry-A_gmean']

#Map the values to the other column 
#All data
s_subset['ZF37_0.5_mGL'] = s_subset['bioreplicate'].map(map_mGLnorm).astype(float)
s_subset['mGL-A_gmean norm ZF37_0.5'] = (s_subset['mGL-A_gmean']) / (s_subset['ZF37_0.5_mGL'])
s_subset['ZF37_0.5_mCherry'] = s_subset['bioreplicate'].map(map_mChnorm).astype(float)
s_subset['mCherry-A_gmean norm ZF37_0.5'] = (s_subset['mCherry-A_gmean']) / (s_subset['ZF37_0.5_mCherry'])
# mGL positive only 
s2_subset['ZF37_0.5_mGL'] = s2_subset['bioreplicate'].map(map_mGLnorm2).astype(float)
s2_subset['mGL-A_gmean norm ZF37_0.5'] = (s2_subset['mGL-A_gmean']) / (s2_subset['ZF37_0.5_mGL'])
s2_subset['ZF37_0.5_mCherry'] = s2_subset['bioreplicate'].map(map_mChnorm2).astype(float)
s2_subset['mCherry-A_gmean norm ZF37_0.5'] = (s2_subset['mCherry-A_gmean']) / (s2_subset['ZF37_0.5_mCherry'])


##############
# Plot normalzied data 
################

# both normalized
#only get the relevant part of the s dataframe
data_now = s_subset
x= 'mCherry-A_gmean norm ZF37_0.5'
hue = 'ZF37_str'
hue_order = pd.array(['0.0', '0.00781','0.01563','0.03125','0.0625','0.125', '0.25','0.5', "1.0"])
palette = 'gist_heat'
style = 'reporter'
xlim = [0.01,5]
ylim = [0.04,200]
yscale = 'log'
stat_list = ['mGL-A_gmean norm ZF37_0.5']
xlabel = 'mCherry-A gmean norm.'
ylabel = 'mGL-A gmean norm.'
fontsize = 15; 
xsize = 4; ysize=xsize

for stat in stat_list:
  y = stat
plottitle = 'Scatterplot both norm 0.5x'
plt.figure(figsize=(xsize,ysize))
g = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize)
g.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

#If want to add in promoter part -- comment out to run the next plot
# plottitle = 'Scatterplot w prom 0.5xNorm'
# data_now = s_promoters37_normEF1a
# hue = 'ZF37'; hue_order = pd.array( ['no ZF', 'pTET112 hPGK.d','pTET EF1a 0.125x','pTET111 duplex UBC.d','pTET EF1a 1x','pTET113 CMV'])
# palette = 'winter'
# x = 'mCherry-A_gmean norm ZF37_EF1a'; y = 'mGL-A_gmean norm ZF37_EF1a'
# k = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
#                  plottitle=plottitle, style=style, hue_order=hue_order,
#                  ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
#                  fontsize=fontsize, labelsize=fontsize)
# k.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')
xlim = [0.04,5]
plottitle = 'Scatterplot both norm 0.5x mGLpos' 
data_now = s2_subset[ s2_subset['ZF37_str']!= '0.0']
plt.figure(figsize=(xsize,ysize))
palette = ['black','gray']; plottitle = plottitle + ' gray';hue= 'reporter'; hue_order = None; 
h = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize)
h.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')

#Overaly plot of the different promoters 
plottitle = 'Scatterplot w prom 0.5xNorm mGLpos' + '2'
hue = 'ZF37'; hue_order = pd.array( [ 'pTET112 hPGK.d','pTET111 duplex UBC.d','pTET EF1a 1x','pTET113 CMV'])
data_now = s_promoters37_normEF1a[s_promoters37_normEF1a['ZF37'].isin(hue_order)]
palette = 'gist_heat'
x = 'mCherry-A_gmean norm ZF37_EF1a'; y = 'mGL-A_gmean norm ZF37_EF1a'
k = scatter_plot(data=data_now, x=x, y=y, hue=hue, palette = palette, 
                 plottitle=plottitle, style=style, hue_order=hue_order,
                 ylim=ylim, xlim=xlim, s= 50, xlabel=xlabel, ylabel=ylabel, 
                 fontsize=fontsize, labelsize=fontsize)
k.figure.savefig(figure_folder + plottitle + '.svg',dpi=300,bbox_inches='tight')



In [ ]:
display( s_promoters37_normEF1a[s_promoters37_normEF1a['ZF37']=='pTET EF1a 1x'])

Assessing significance in difference in modeling parameters

In [ ]:
params = {
    'Cre': ['noCre','noCre','noCre','noCre','noCre',
                     'Cre','Cre','Cre','Cre','Cre'],
    'k_cat': [0.94033, 0.89261, 0.84912, 1.2221, 1.06148,
              10.47357, 9.30979, 9.996932, 21.64241, 22.69283]
}

df_params=pd.DataFrame(params)

print(df_params)

In [ ]:
plottitle='ZF37 k_cat +/-Cre per biorep'

g = summary_plot( x='Cre', y='k_cat',data=df_params, hue='Cre',
                    x_rot=45, plottitle=plottitle, type='barplot')